# Tool Wrappers

> Automatically generate DSPy modules from our component registry.

This module takes the component registry and generates fully-typed DSPy modules for each tool. It converts the component metadata into actual DSPy Module classes with proper signatures and documentation.

In [ ]:
#| default_exp wrappers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import re
import inspect
import dspy
from cogitarelink_dspy.components import  COMPONENTS

/Users/cvardema/dev/git/LA3D/cogitarelink/cogitarelink-dspy/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
def parse_signature(sig_str):
    """Parse a signature string like 'foo(a:str, b:int) -> str' into parameter names,
    types, and return type.
    
    Handles complex types like List[str], Dict[str, Any], etc.
    
    Args:
        sig_str: A string in the format "function_name(param1:type, param2:type) -> return_type"
                 or just "param1:type, param2:type) -> return_type"
                 
    Returns:
        tuple: (list of (name, type) tuples for parameters, return_type)
    """
    # Extract the part inside parentheses if it's a full function signature
    if '(' in sig_str:
        # Handle function name and everything inside parentheses
        fn_part = sig_str.split('(', 1)
        params_str = fn_part[1].split(')', 1)[0]
    else:
        params_str = sig_str.split(')', 1)[0]
        
    # Extract return type if present
    return_type = None
    if ' -> ' in sig_str:
        return_type = sig_str.split(' -> ')[1].strip()
    
    # Parse parameters - handle complex types with brackets which may contain commas
    params = []
    if params_str.strip():
        # First, handle nested types with braces that might contain commas
        processed_params = []
        param_buffer = ""
        bracket_level = 0
        
        for char in params_str:
            if char == ',' and bracket_level == 0:
                processed_params.append(param_buffer.strip())
                param_buffer = ""
            else:
                param_buffer += char
                if char == '[' or char == '{':
                    bracket_level += 1
                elif char == ']' or char == '}':
                    bracket_level -= 1
        
        # Add the last parameter if buffer is not empty
        if param_buffer.strip():
            processed_params.append(param_buffer.strip())
        
        # Now parse each parameter
        for param in processed_params:
            param = param.strip()
            if ':' in param:
                name, type_hint = param.split(':', 1)  # Split on first colon only
                params.append((name.strip(), type_hint.strip()))
            else:
                # If no type hint, default to str
                params.append((param.strip(), 'str'))
    
    return params, return_type

In [ ]:
# Test the signature parser with more complex cases
test_signatures = [
    # Basic cases
    ("forward(message:str)", [("message", "str")], None),
    ("load(source:str) -> dict", [("source", "str")], "dict"),
    ("validate(subject:str, predicate:str, object:str)", 
     [("subject", "str"), ("predicate", "str"), ("object", "str")], None),
    ("verify(graph_id:str, signature:str) -> bool", 
     [("graph_id", "str"), ("signature", "str")], "bool"),
     
    # Complex types
    ("fetch(urls:List[str]) -> Dict[str, Any]",
     [("urls", "List[str]")], "Dict[str, Any]"),
    ("process(data:Dict[str, List[int]], options:Optional[Dict[str, bool]]=None) -> Tuple[int, str]",
     [("data", "Dict[str, List[int]]"), ("options", "Optional[Dict[str, bool]]=None")], "Tuple[int, str]"),
     
    # Edge cases
    ("complex_func(a:int, b:List[Tuple[str, int]], c:Dict[str, List[Dict[str, Any]]]) -> bool",
     [("a", "int"), ("b", "List[Tuple[str, int]]"), ("c", "Dict[str, List[Dict[str, Any]]]")], "bool")
]

for sig_str, expected_params, expected_return in test_signatures:
    params, return_type = parse_signature(sig_str)
    assert params == expected_params, f"For {sig_str}, expected params {expected_params}, got {params}"
    if expected_return:
        assert return_type == expected_return, f"For {sig_str}, expected return type {expected_return}, got {return_type}"
    
print("All signature parser tests passed!")

All signature parser tests passed!


In [ ]:
#| export
def make_tool_wrappers(registry=COMPONENTS):
    """Generate DSPy Module classes for each tool in the registry.
    
    Args:
        registry: Dictionary of component definitions with layer, tool, doc, and calls fields
                 
    Returns:
        list: A list of DSPy Module classes, one for each component
    """
    tools = []
    
    for name, meta in registry.items():
        # Create a function to define tool class - prevents closure capture bug
        def create_tool_class(name, meta):
            # Get call signature from the component metadata
            call_sig = meta["calls"]
            params, return_type = parse_signature(call_sig)
            
            # Create signature string for DSPy - handle complex types by simplifying
            # Replace complex types with simpler types that DSPy can handle
            simplified_params = []
            for param_name, param_type in params:
                # Simplify complex types to basic types DSPy can understand
                if 'Entity' in param_type or 'Union' in param_type:
                    simplified_type = 'str'
                elif any(t in param_type for t in ['List', 'Dict', 'Tuple']):
                    simplified_type = 'dict'
                else:
                    simplified_type = param_type
                simplified_params.append((param_name, simplified_type))
            
            # Use simplified parameters for DSPy signature
            param_sig = ", ".join(f"{p[0]}:{p[1]}" for p in simplified_params)
            output_type = "str" if return_type and "Entity" in return_type else (return_type or "output")
            signature_str = f"{param_sig} -> {output_type}"
            
            # Documentation for the tool and build its signature
            class_doc = f"{meta['doc']} [Layer: {meta['layer']}]"
            # Generate the DSPy Signature class and instantiate it
            try:
                sig_cls = dspy.Signature(signature_str, "Tool wrapper signature")
            except Exception:
                # Fallback to an empty signature if parsing fails
                sig_cls = dspy.Signature({}, "Tool wrapper signature")
            # Instantiate the signature, falling back to Echo.signature if needed
            try:
                sig_instance = sig_cls()
            except Exception:
                # Fallback to an empty Signature subclass instance to avoid required fields
                class EmptySig(dspy.Signature):
                    """Empty Signature with no fields"""
                    pass
                sig_instance = EmptySig()
            # Define the DSPy Module class for this tool
            class ToolWrapper(dspy.Module):
                """Placeholder docstring that will be replaced."""
                # Assign the signature instance directly
                signature = sig_instance
                
                def forward(self, **kwargs):
                    """Forward the call to the actual implementation."""
                    try:
                        # Attempt to import the actual module
                        module_path = meta['module']
                        components = module_path.split('.')
                        
                        # Handle different module structures
                        # Case 1: function directly in module (like cogitarelink.verify.validator.validate_entity)
                        if len(components) > 1:
                            # Import parent module first
                            parent_module_path = '.'.join(components[:-1])
                            import importlib
                            parent_module = importlib.import_module(parent_module_path)
                            
                            try:
                                # Try to get the function from the parent module
                                function_name = call_sig.split('(')[0]
                                func = getattr(parent_module, function_name)
                                return func(**kwargs)
                            except (AttributeError, IndexError):
                                # Try to get the class if function not found
                                class_name = components[-1].capitalize()
                                class_obj = getattr(parent_module, class_name)
                                instance = class_obj()
                                
                                # Get method name from call_sig
                                method_name = call_sig.split('(')[0]
                                method = getattr(instance, method_name)
                                return method(**kwargs)
                        
                        # Case 2: Class needs to be instantiated first
                        else:
                            # Import the module
                            import importlib
                            module = importlib.import_module(module_path)
                            
                            # Get the class name from the component name
                            class_name = name
                            class_obj = getattr(module, class_name)
                            
                            # Create an instance
                            instance = class_obj()
                            
                            # Get the method name from call signature
                            method_name = call_sig.split('(')[0]
                            method = getattr(instance, method_name)
                            
                            # Call the method
                            return method(**kwargs)
                    
                    except Exception as e:
                        # Just log and return a fallback response for now
                        print(f"Error calling {meta['tool']}: {e}")
                        return f"Mock result from {meta['tool']} with args: {kwargs}"
            
            # Set proper class name and docstring
            ToolWrapper.__doc__ = class_doc
            ToolWrapper.__name__ = meta['tool']
            ToolWrapper.__qualname__ = meta['tool']
            
            # Store original parameter info as class attributes for reference
            ToolWrapper.original_params = params
            ToolWrapper.original_return_type = return_type
            
            # Add layer as a class attribute for easier access
            ToolWrapper.layer = meta['layer']
            ToolWrapper.module_path = meta.get('module', '')
            
            return ToolWrapper
        
        # Create the tool class and add to tools list
        tool_class = create_tool_class(name, meta)
        tools.append(tool_class)
    
    return tools

In [ ]:
#| export
# Initialize tool wrappers at import time
TOOLS = make_tool_wrappers()

def get_tools():
    """Get or initialize the tool wrappers.
    
    Returns:
        list: A list of DSPy Module classes, one for each component
    """
    global TOOLS
    if TOOLS is None:
        TOOLS = make_tool_wrappers()
    return TOOLS

def get_tool_by_name(tool_name):
    """Find a specific tool by its name.
    
    Args:
        tool_name (str): The name of the tool to find
        
    Returns:
        class or None: The tool class if found, None otherwise
    """
    tools = get_tools()
    for tool in tools:
        if tool.__name__ == tool_name:
            return tool
    return None

# Helper function to organize tools by layer
def group_tools_by_layer(tools=None):
    """Group the generated tools by their semantic layer.
    
    Args:
        tools: List of tool classes to group. If None, uses get_tools().
        
    Returns:
        dict: A dictionary with layers as keys and lists of tools as values
    """
    if tools is None:
        tools = get_tools()
        
    result = {}
    for tool in tools:
        # Get layer directly from the class attribute we added
        layer = tool.layer
        if layer not in result:
            result[layer] = []
        result[layer].append(tool)
    return result

/Users/cvardema/dev/git/LA3D/cogitarelink/cogitarelink-dspy/.venv/lib/python3.13/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "dict" in "StringSignature" shadows an attribute in parent "Signature"
  warnings.warn(
/Users/cvardema/dev/git/LA3D/cogitarelink/cogitarelink-dspy/.venv/lib/python3.13/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "signature" in "StringSignature" shadows an attribute in parent "Signature"
  warnings.warn(


In [ ]:
# Test the tool wrapper generation functionality
try:
    # Generate the tools
    tools = make_tool_wrappers()
    assert len(tools) == len(COMPONENTS), f"Expected {len(COMPONENTS)} tools, got {len(tools)}"
    
    # Test a sample tool
    for tool_class in tools[:1]:  # Just check the first tool
        print(f"Tool name: {tool_class.__name__}")
        print(f"Documentation: {tool_class.__doc__}")
        print(f"Layer: {tool_class.layer}")
        print(f"Module path: {tool_class.module_path}")
        
        # Create an instance and test it
        instance = tool_class()
        sample_args = {"message": "Testing tool wrapper"} if "message" in str(tool_class.signature) else {}
        print(f"\nCalling with: {sample_args}")
        try:
            result = instance(**sample_args)
            print(f"Result: {result}")
        except Exception as e:
            print(f"Error calling tool: {e}")
except Exception as e:
    print(f"Error testing tool wrappers: {e}")
    print("This is expected during notebook testing without all dependencies.")

Tool name: EchoMessage
Documentation: Simply echoes the input message back. [Layer: Utility]
Layer: Utility
Module path: cogitarelink.utils

Calling with: {}
Error calling EchoMessage: module 'cogitarelink' has no attribute 'Utils'
Result: Mock result from EchoMessage with args: {}


## Using the Generated Tools

Each tool is a DSPy Module class that can be instantiated and used in a DSPy pipeline. Here's how to use the tools:

1. **Individual Tool Usage**:
   - Instantiate a specific tool using its class
   - Call it with the appropriate parameters as defined in its signature

2. **Layer-Based Tool Selection**:
   - Use the `group_tools_by_layer()` function to organize tools by layer
   - Select tools from the appropriate layer based on the user's query
   
3. **Integration with DSPy Agent**:
   - Pass the entire `TOOLS` list to a `dspy.StructuredAgent`
   - The agent will be able to discover and use the tools based on their signatures and documentation

In [ ]:
# Example: Create an agent that can use our tools
def create_semantic_agent(lm=None):
    """Create a semantic agent with the generated tools.
    
    This function will eventually be moved to its own module. It's shown here
    as an illustration of how the tools will be used in a DSPy agent.
    """
    import dspy
    from cogitarelink_dspy.core import default_lm
    
    # Use the LLM from core if none is provided
    lm = lm or default_lm
    
    # Create a system prompt that explains the 4-layer architecture
    system_prompt = """
    You are a Semantic-Web agent that reasons over a 4-layer architecture:
    1. Context - Working with JSON-LD contexts and namespaces
    2. Ontology - Using vocabularies and ontology terms
    3. Rules - Applying validation rules and shapes
    4. Instances - Managing actual data instances
    5. Verification - Verifying and signing graph data
    
    Every tool is tagged with its PRIMARY layer. When answering a user question,
    pick the tool from the HIGHEST layer that suffices to answer the question.
    """
    
    semantic_lm = dspy.LM(lm.model, system=system_prompt) if lm else None
    
    # Create a StructuredAgent with our tools
    agent = dspy.StructuredAgent(
        tools=get_tools(),  # Get tools using the getter function
        lm=semantic_lm
    )
    
    return agent

# This will be implemented in a future notebook focused on the agent
# agent = create_semantic_agent() 
# result = agent.query("Load the schema.org context")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()